In [7]:
from torch import nn
from torch import sum, mean
import torch
from common.dataset import SegmentationDataset
from common.transformations import (
    SegCompose,
    SegCrop,
    SegRandomHorizontalFlip,
    SegRandomVerticalFlip,
)
from os.path import join
from torch import cuda, device, save
from os.path import join
from torch.nn import CrossEntropyLoss, Module
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torchvision.transforms import (
    Compose,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    Resize,
    ToTensor,)


TRANSVERSE_TRAIN_IMG_PATH = join("data", "seg_train_small", "imgs", "trans")
TRANSVERSE_TRAIN_LABELS_PATH = join("data", "seg_train_small", "labels", "trans")
# transformations
TRANSFORMATIONS_SEG = SegCompose(
    [
        SegCrop(default_t=15),
    ]
)
TRANSFORMATIONS_TORCH = Compose(
    [
        # original| Resize((512, 512)),
        Resize((128, 128)),
        ToTensor(),
    ]
)
val_dataset = SegmentationDataset(
        TRANSVERSE_TRAIN_IMG_PATH, TRANSVERSE_TRAIN_LABELS_PATH, TRANSFORMATIONS_SEG, TRANSFORMATIONS_TORCH
    )

img = val_dataset[0][0]



In [15]:
img

tensor([[[0.1608, 0.1647, 0.1608,  ..., 0.1647, 0.1725, 0.1725],
         [0.1725, 0.1608, 0.1451,  ..., 0.1765, 0.1922, 0.1922],
         [0.1725, 0.1686, 0.1451,  ..., 0.1882, 0.1882, 0.2000],
         ...,
         [0.0941, 0.1059, 0.1059,  ..., 0.0588, 0.0510, 0.0392],
         [0.0941, 0.0980, 0.0902,  ..., 0.0431, 0.0431, 0.0314],
         [0.0941, 0.0902, 0.0941,  ..., 0.0235, 0.0196, 0.0157]],

        [[0.1686, 0.1686, 0.1686,  ..., 0.1725, 0.1765, 0.1765],
         [0.1765, 0.1647, 0.1529,  ..., 0.1843, 0.1961, 0.1961],
         [0.1804, 0.1765, 0.1490,  ..., 0.1961, 0.1961, 0.2078],
         ...,
         [0.0980, 0.1098, 0.1098,  ..., 0.0588, 0.0549, 0.0431],
         [0.0980, 0.1020, 0.0941,  ..., 0.0431, 0.0431, 0.0353],
         [0.0980, 0.0941, 0.0980,  ..., 0.0235, 0.0196, 0.0157]],

        [[0.1725, 0.1765, 0.1765,  ..., 0.1765, 0.1843, 0.1843],
         [0.1843, 0.1725, 0.1608,  ..., 0.1922, 0.2039, 0.2039],
         [0.1882, 0.1843, 0.1569,  ..., 0.2039, 0.2000, 0.

In [33]:
img_mean = torch.std(val_dataset[0][0], dim=(1,2))
# img.shape
img_mean

tensor([0.1567, 0.1637, 0.1709])

In [18]:
# torch.max(val_dataset[0][0], dim=(1,2))
torch.min(val_dataset[0][0])

tensor(0.0039)

In [19]:

# Define mean and std deviation values
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

# Standardize the image
standardized_image = (img - mean[:, None, None]) / std[:, None, None]
standardized_image


tensor([[[-1.4158, -1.3987, -1.4158,  ..., -1.3987, -1.3644, -1.3644],
         [-1.3644, -1.4158, -1.4843,  ..., -1.3473, -1.2788, -1.2788],
         [-1.3644, -1.3815, -1.4843,  ..., -1.2959, -1.2959, -1.2445],
         ...,
         [-1.7069, -1.6555, -1.6555,  ..., -1.8610, -1.8953, -1.9467],
         [-1.7069, -1.6898, -1.7240,  ..., -1.9295, -1.9295, -1.9809],
         [-1.7069, -1.7240, -1.7069,  ..., -2.0152, -2.0323, -2.0494]],

        [[-1.2829, -1.2829, -1.2829,  ..., -1.2654, -1.2479, -1.2479],
         [-1.2479, -1.3004, -1.3529,  ..., -1.2129, -1.1604, -1.1604],
         [-1.2304, -1.2479, -1.3704,  ..., -1.1604, -1.1604, -1.1078],
         ...,
         [-1.5980, -1.5455, -1.5455,  ..., -1.7731, -1.7906, -1.8431],
         [-1.5980, -1.5805, -1.6155,  ..., -1.8431, -1.8431, -1.8782],
         [-1.5980, -1.6155, -1.5980,  ..., -1.9307, -1.9482, -1.9657]],

        [[-1.0376, -1.0201, -1.0201,  ..., -1.0201, -0.9853, -0.9853],
         [-0.9853, -1.0376, -1.0898,  ..., -0

In [30]:
imagenet_mean = torch.tensor([0.485, 0.456, 0.406])
imagenet_std = torch.tensor([0.229, 0.224, 0.225])

# Standardize the image tensor to have ImageNet mean and std
standardized_image = (img - imagenet_mean[:, None, None]) / imagenet_std[:, None, None]
standardized_image

tensor([[[-1.4158, -1.3987, -1.4158,  ..., -1.3987, -1.3644, -1.3644],
         [-1.3644, -1.4158, -1.4843,  ..., -1.3473, -1.2788, -1.2788],
         [-1.3644, -1.3815, -1.4843,  ..., -1.2959, -1.2959, -1.2445],
         ...,
         [-1.7069, -1.6555, -1.6555,  ..., -1.8610, -1.8953, -1.9467],
         [-1.7069, -1.6898, -1.7240,  ..., -1.9295, -1.9295, -1.9809],
         [-1.7069, -1.7240, -1.7069,  ..., -2.0152, -2.0323, -2.0494]],

        [[-1.2829, -1.2829, -1.2829,  ..., -1.2654, -1.2479, -1.2479],
         [-1.2479, -1.3004, -1.3529,  ..., -1.2129, -1.1604, -1.1604],
         [-1.2304, -1.2479, -1.3704,  ..., -1.1604, -1.1604, -1.1078],
         ...,
         [-1.5980, -1.5455, -1.5455,  ..., -1.7731, -1.7906, -1.8431],
         [-1.5980, -1.5805, -1.6155,  ..., -1.8431, -1.8431, -1.8782],
         [-1.5980, -1.6155, -1.5980,  ..., -1.9307, -1.9482, -1.9657]],

        [[-1.0376, -1.0201, -1.0201,  ..., -1.0201, -0.9853, -0.9853],
         [-0.9853, -1.0376, -1.0898,  ..., -0

In [31]:
torch.mean(standardized_image, dim = (1,2))

tensor([-1.4490, -1.3232, -1.0668])